# Load-Dependent Example 3: Multi-Class Load Dependence with PS Scheduling

This example demonstrates load-dependent queues with multiple job classes and PS (Processor Sharing) scheduling.

**Features:**
- Two closed classes: Class1 (4 jobs), Class2 (2 jobs)
- Three stations: Delay + two PS queues with load dependence
- Each queue has 3 servers with load dependence: min(total_jobs, servers)
- Compares load-independent model vs load-dependent model
- Demonstrates different solver methods: CTMC, NC, MVA

In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Model parameters
N = 4  # number of jobs in Class1
c = 3  # number of servers per queue

# Create load-independent model for comparison
model = Network('model')
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)
node3 = Queue(model, 'Queue2', SchedStrategy.PS)

# Create job classes
jobclass1 = ClosedClass(model, 'Class1', N, node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N // 2, node1, 0)  # N/2 = 2 jobs

# Set service times
node1.set_service(jobclass1, Exp.fitMean(1.0))  # mean = 1.0
node1.set_service(jobclass2, Exp.fitMean(2.0))  # mean = 2.0

node2.set_service(jobclass1, Exp.fitMean(1.5))  # mean = 1.5
node2.set_service(jobclass2, Exp.fitMean(2.5))  # mean = 2.5
node2.set_number_of_servers(c)

node3.set_service(jobclass1, Exp.fitMean(3.5))  # mean = 3.5
node3.set_service(jobclass2, Exp.fitMean(4.5))  # mean = 4.5
node3.set_number_of_servers(c)

# Create routing matrix (serial routing for each class)
P = model.init_routing_matrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)
model.link(P)

In [3]:
# Solve load-independent model with MVA
solver_mva = SolverMVA(model, method='exact')
msT = solver_mva.get_avg_table()

MVA [method: exact, lang: java, env: 17.0.15] completed in 0.000099s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.5467  0.5467  1.0000  1.0000  0.5467  0.5467
1   Delay   Class2  0.3694  0.3694  2.0000  2.0000  0.1847  0.1847
2  Queue1   Class1  0.8564  0.2734  1.5664  1.5664  0.5467  0.5467
3  Queue1   Class2  0.4808  0.1539  2.6027  2.6027  0.1847  0.1847
4  Queue2   Class1  2.5969  0.6379  4.7498  4.7498  0.5467  0.5467
5  Queue2   Class2  1.1498  0.2771  6.2244  6.2244  0.1847  0.1847


In [4]:
# Create load-dependent model
ldmodel = Network('ldmodel')
node1 = Delay(ldmodel, 'Delay')
node2 = Queue(ldmodel, 'Queue1', SchedStrategy.PS)
node3 = Queue(ldmodel, 'Queue2', SchedStrategy.PS)

# Create job classes
jobclass1 = ClosedClass(ldmodel, 'Class1', N, node1, 0)
jobclass2 = ClosedClass(ldmodel, 'Class2', N // 2, node1, 0)

# Set service times (same as load-independent model)
node1.set_service(jobclass1, Exp.fitMean(1.0))
node1.set_service(jobclass2, Exp.fitMean(2.0))

node2.set_service(jobclass1, Exp.fitMean(1.5))
node2.set_service(jobclass2, Exp.fitMean(2.5))

node3.set_service(jobclass1, Exp.fitMean(3.5))
node3.set_service(jobclass2, Exp.fitMean(4.5))

# Set load dependence: min(total_jobs, c) servers
total_jobs = N + N // 2  # 4 + 2 = 6 total jobs
alpha = np.minimum(np.arange(1, total_jobs + 1), c)  # min(1:6, 3) = [1,2,3,3,3,3]
node2.set_load_dependence(alpha)
node3.set_load_dependence(alpha)

# Create routing matrix
P = ldmodel.init_routing_matrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)
ldmodel.link(P)

In [5]:
# Solve with CTMC (exact solution)
solver_ctmc = SolverCTMC(ldmodel)
lldAvgTableCTMC = solver_ctmc.get_avg_table()

CTMC [method: default, lang: java, env: 17.0.15] completed in 0.410000s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.5467  0.5467  1.0000  1.0000  0.5467  0.5467
1   Delay   Class2  0.3694  0.3694  2.0000  2.0000  0.1847  0.1847
2  Queue1   Class1  0.8564  0.4739  1.5664  1.5664  0.5467  0.5467
3  Queue1   Class2  0.4808  0.2713  2.6027  2.6027  0.1847  0.1847
4  Queue2   Class1  2.5969  0.6941  4.7498  4.7498  0.5467  0.5467
5  Queue2   Class2  1.1498  0.2980  6.2244  6.2244  0.1847  0.1847


In [6]:
# Solve with NC (Normalizing Constant) - Exact method
solver_nc_exact = SolverNC(ldmodel, method='exact')
lldAvgTableNC = solver_nc_exact.get_avg_table()

java.lang.RuntimeException: NC solver cannot provide exact solutions for open or mixed queueing networks. Remove the 'exact' option.
	at jline.solvers.nc.analyzers.Solver_ncld_analyzerKt.solver_ncld_analyzer(Solver_ncld_analyzer.kt:23)
	at jline.solvers.nc.SolverNC.runAnalyzer(SolverNC.java:416)
	at jline.solvers.NetworkSolver.getAvg(NetworkSolver.java:216)
	at jline.solvers.NetworkSolver.getAvgTable(NetworkSolver.java:1659)


java.lang.RuntimeException: java.lang.RuntimeException: Unable to compute results and therefore unable to print AvgTable.

In [ ]:
# Solve with NC - Reduction heuristic (RD) method
solver_nc_rd = SolverNC(ldmodel, method='rd')
lldAvgTableRD = solver_nc_rd.get_avg_table()

In [ ]:
# Solve with NC - NRP and NRL methods
solver_nc_nrp = SolverNC(ldmodel, method='nrp')
lldAvgTableNRP = solver_nc_nrp.get_avg_table()

In [ ]:

solver_nc_nrl = SolverNC(ldmodel, method='nrl')
lldAvgTableNRL = solver_nc_nrl.get_avg_table()

In [ ]:
# Solve with MVA for load-dependent model
solver_mva_ld = SolverMVA(ldmodel, method='exact')
lldAvgTableMVALD = solver_mva_ld.get_avg_table()

In [ ]:

solver_mva_qd = SolverMVA(ldmodel, method='qd')
lldAvgTableQD = solver_mva_qd.get_avg_table()

## Results Analysis

This example demonstrates the effect of load dependence on queue performance:

1. **Load-Independent Model**: Each queue has a fixed number of servers (3) regardless of population
2. **Load-Dependent Model**: Number of active servers depends on total population: min(population, 3)

The load dependence affects system performance by:
- Limiting the effective service capacity when population is low
- Providing full service capacity only when enough jobs are present
- This creates more realistic modeling of servers that scale with demand

Different solvers (CTMC, NC, MVA) provide various numerical methods for solving the same model, allowing comparison of accuracy and computational efficiency.